In [1]:
import numpy as np
import pandas as pd
import scipy.stats as sps

import matplotlib.pyplot as plt
import seaborn as sns

from tqdm.auto import tqdm

%matplotlib inline
%config InlineBackend.figure_format = 'svg'
sns.set(font_scale=1.3, style='darkgrid', palette='Set2')

In [2]:
from enum import Enum
from collections import defaultdict

import brian2 as b2
import gym
from scipy.optimize import differential_evolution

import Walker750

import warnings
from warnings import filterwarnings
# filterwarnings(action='ignore')
filterwarnings(action='ignore', category=DeprecationWarning, module='.*brian2.*')
filterwarnings(action='ignore', category=DeprecationWarning, 
               message='.*the imp module is deprecated in favour of importlib and slated for.*')
filterwarnings(action='ignore', category=DeprecationWarning, 
               message='.*WARN: Initializing wrapper in old step API which returns one bool instead.*')
filterwarnings(action='ignore', category=DeprecationWarning,
               message='.*WARN: Initializing environment in old step API which returns one bool.*')
filterwarnings(action='ignore', category=UserWarning, 
               message='.*WARN: We recommend you to use a symmetric and normalized Box action space.*')

In [3]:
params0 = [-0.607, -0.311, -1.649, -1.934, # w1-w8
           0.285, 0.143, 0.302, 0.151,     # tau
           0.124, 0.770,                   # a
           0.805, 3.078, -2.120]           # u0, b, w

#          w1-w8                           tau
bounds = [(-4, 0) for _ in range(4)] + [(0.04, 1) for _ in range(4)] +\
         [(0.03, 1) for _ in range(2)] + [(0, 2), (0, 4), (-4, 0)]
#             a                              u0      b       w

n_iter = 150
pbar = tqdm(total=n_iter)

def callback(x, convergence):
    print(f'params = {x}; convergence = {convergence}\n')
    with open('params.log', 'a') as f:
        f.write(f'params = {x}; convergence = {convergence}\n')
    pbar.update(1)

res = differential_evolution(Walker.calc_reward, bounds, maxiter=n_iter, callback=callback,
                             disp=True, x0=params0, updating='deferred', workers=9)
pbar.close()

# 16 - 45min

  0%|          | 0/150 [00:00<?, ?it/s]

/Users/aleks311001/mambaforge/envs/neuro/lib/python3.10/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  from imp import reload
/Users/aleks311001/mambaforge/envs/neuro/lib/python3.10/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  from imp import reload
/Users/aleks311001/mambaforge/envs/neuro/lib/python3.10/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  from imp import reload
/Users/aleks311001/mambaforge/envs/neuro/lib/python3.10/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in 

clang-13: error: the clang compiler does not support '-march=native'
WARNING    Cannot use Cython, a test compilation failed: command '/Users/aleks311001/mambaforge/envs/neuro/bin/arm64-apple-darwin20.0.0-clang' failed with exit code 1 (CompileError) [brian2.codegen.runtime.cython_rt.cython_rt.failed_compile_test]
INFO       Cannot use compiled code, falling back to the numpy code generation target. Note that this will likely be slower than using compiled code. Set the code generation to numpy manually to avoid this message:
prefs.codegen.target = "numpy" [brian2.devices.device.codegen_fallback]
clang-13: error: the clang compiler does not support '-march=native'
WARNING    Cannot use Cython, a test compilation failed: command '/Users/aleks311001/mambaforge/envs/neuro/bin/arm64-apple-darwin20.0.0-clang' failed with exit code 1 (CompileError) [brian2.codegen.runtime.cython_rt.cython_rt.failed_compile_test]
INFO       Cannot use compiled code, falling back to the numpy code generation ta

differential_evolution step 1: f(x)= -4.39393
params = [-3.59501631 -1.96946159 -1.93599384 -2.98718828  0.06744325  0.76629448
  0.77100639  0.84867654  0.245692    0.20664682  1.66493787  3.04727016
 -3.26314092]; convergence = 0.021701426301044484



WARNING    neurongroup's variable 'u2' has NaN, very large values, or encountered an error in numerical integration. This is usually a sign that an unstable or invalid integration method was chosen. [brian2.groups.group.invalid_values]
WARNING    neurongroup's variable 'u1' has NaN, very large values, or encountered an error in numerical integration. This is usually a sign that an unstable or invalid integration method was chosen. [brian2.groups.group.invalid_values]
WARNING    neurongroup_2's variable 'u1' has NaN, very large values, or encountered an error in numerical integration. This is usually a sign that an unstable or invalid integration method was chosen. [brian2.groups.group.invalid_values]
WARNING    neurongroup_2's variable 'u2' has NaN, very large values, or encountered an error in numerical integration. This is usually a sign that an unstable or invalid integration method was chosen. [brian2.groups.group.invalid_values]
WARNING    neurongroup's variable 'v2' has NaN, very

...

differential_evolution step 97: f(x)= -20.0617
params = [-3.15788288 -0.41961284 -2.6157932  -2.67779105  0.07335032  0.10212809
  0.64912344  0.64802246  0.20336929  0.99670978  0.47047603  2.30910493
 -2.91164797]; convergence = 0.036795447955820355



WARNING    neurongroup_3's variable 'u1' has NaN, very large values, or encountered an error in numerical integration. This is usually a sign that an unstable or invalid integration method was chosen. [brian2.groups.group.invalid_values]
WARNING    neurongroup_3's variable 'u2' has NaN, very large values, or encountered an error in numerical integration. This is usually a sign that an unstable or invalid integration method was chosen. [brian2.groups.group.invalid_values]
WARNING    neurongroup_3's variable 'v2' has NaN, very large values, or encountered an error in numerical integration. This is usually a sign that an unstable or invalid integration method was chosen. [brian2.groups.group.invalid_values]
WARNING    neurongroup_3's variable 'v1' has NaN, very large values, or encountered an error in numerical integration. This is usually a sign that an unstable or invalid integration method was chosen. [brian2.groups.group.invalid_values]


differential_evolution step 98: f(x)= -20.0617
params = [-3.15788288 -0.41961284 -2.6157932  -2.67779105  0.07335032  0.10212809
  0.64912344  0.64802246  0.20336929  0.99670978  0.47047603  2.30910493
 -2.91164797]; convergence = 0.03691051258042287

differential_evolution step 99: f(x)= -20.0617
params = [-3.15788288 -0.41961284 -2.6157932  -2.67779105  0.07335032  0.10212809
  0.64912344  0.64802246  0.20336929  0.99670978  0.47047603  2.30910493
 -2.91164797]; convergence = 0.037509828930223

differential_evolution step 100: f(x)= -20.0617
params = [-3.15788288 -0.41961284 -2.6157932  -2.67779105  0.07335032  0.10212809
  0.64912344  0.64802246  0.20336929  0.99670978  0.47047603  2.30910493
 -2.91164797]; convergence = 0.03766381237025796

differential_evolution step 101: f(x)= -20.0617
params = [-3.15788288 -0.41961284 -2.6157932  -2.67779105  0.07335032  0.10212809
  0.64912344  0.64802246  0.20336929  0.99670978  0.47047603  2.30910493
 -2.91164797]; convergence = 0.03817650253

Process SpawnPoolWorker-6:
Process SpawnPoolWorker-7:
Process SpawnPoolWorker-8:
Process SpawnPoolWorker-5:
Process SpawnPoolWorker-3:
Process SpawnPoolWorker-4:
Process SpawnPoolWorker-2:
Process SpawnPoolWorker-1:
Process SpawnPoolWorker-9:


KeyboardInterrupt: 

In [ ]:
params0 = [-0.607, -0.311, -1.649, -1.934, # w1-w8
           0.285, 0.143, 0.302, 0.151,     # tau
           0.124, 0.770,                   # a
           0.805, 3.078, -2.120, 1]        # u0, b, w, k_action

#          w1, w4                        w5, w7                           tau
bounds = [(-1, 0) for _ in range(2)] + [(-2.5, -1) for _ in range(2)] + [(0.1, 0.5) for _ in range(4)] +\
         [(0.075, 0.2), (0.5, 1), (0, 2), (0, 4), (-4, 0), (0.3, 8)]
#               a1         a2       u0      b       w       k_action

n_iter = 150
pbar = tqdm(total=n_iter)

def callback(x, convergence):
    print(f'params = {x}; convergence = {convergence}\n')
    with open('params.log', 'a') as f:
        f.write(f'params = {x}; convergence = {convergence}\n')
    pbar.update(1)

res = differential_evolution(Walker750.calc_reward, bounds, maxiter=n_iter, callback=callback,
                             disp=True, x0=params0, updating='deferred', workers=9)
pbar.close()

# 16 - 45min

In [ ]:
params = [-3.08761372, -1.77306303, -1.41658687, -1.46172341, 0.04756068,
          0.17430144, 0.99882352, 0.84377149, 0.20099308, 0.71020467,
          1.46892851, 0.61687972, -1.89211497] # ~-9

In [ ]:
env = gym.make('BipedalWalker-v3')
c = make_controller(res.x)
observation = env.reset()

for _ in range(250):
    y = c.step(observation)
    action = np.array([y[Side.LEFT][Joint.HIP], y[Side.LEFT][Joint.KNEE], 
                       y[Side.RIGHT][Joint.HIP], y[Side.RIGHT][Joint.KNEE]])
    action = np.clip(action, -1, 1)
#     print(action)
    observation, reward, done, _ = env.step(action)
#     print(done, info)
#     print(reward)
    env.render()

env.close()

In [ ]:
plot_all(c)